In [0]:
!pip install keras_vggface

  Stored in directory: /root/.cache/pip/wheels/72/aa/01/eb7baeb2f6e2d2f0d2aabddb5f01d57fa22fbd019ee2799bf5
Successfully built keras-vggface


In [0]:
!pip install keras==2.2.0

     |████████████████████████████████| 307kB 9.1MB/s 
     |████████████████████████████████| 51kB 27.7MB/s 
ERROR: tensorflow 1.13.1 has requirement keras-applications>=1.0.6, but you'll have keras-applications 1.0.2 which is incompatible.
ERROR: tensorflow 1.13.1 has requirement keras-preprocessing>=1.0.5, but you'll have keras-preprocessing 1.0.1 which is incompatible.
  Found existing installation: Keras-Preprocessing 1.0.9
    Uninstalling Keras-Preprocessing-1.0.9:
      Successfully uninstalled Keras-Preprocessing-1.0.9
  Found existing installation: Keras-Applications 1.0.7
    Uninstalling Keras-Applications-1.0.7:
      Successfully uninstalled Keras-Applications-1.0.7
  Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [0]:
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

Using TensorFlow backend.


In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
model = VGGFace(model='resnet50', include_top=False,input_shape=(224,224,3))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_3[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_50 (Activation)      (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

In [0]:
import numpy as np
from keras import losses
from keras import backend as K

def baseline_model():
    x = Input(shape=(1,1,2048))

    # x1_ = Reshape(target_shape=(7*7, 2048))(x1)
    # x2_ = Reshape(target_shape=(7*7, 2048))(x2)
    #
    # x_dot = Dot(axes=[2, 2], normalize=True)([x1_, x2_])
    # x_dot = Flatten()(x_dot)

    x1 = Concatenate(axis=-1)([GlobalMaxPool2D()(x), GlobalAvgPool2D()(x)])
    x2 = Concatenate(axis=-1)([GlobalMaxPool2D()(x), GlobalAvgPool2D()(x)])

    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])

    x = Multiply()([x1, x2])

    x = Concatenate(axis=-1)([x, x3])

    x = Dense(100, activation="relu")(x)
    x = Dropout(0.1)(x)
    out = Dense(1, activation="sigmoid")(x)
    
layer1 = Input(shape=(1,1,2048))

layer2 = GlobalMaxPool2D()(layer1)

model = Model(inputs=layer1,outputs=layer2)

inp = np.random.rand(1,1,1,2048)

model.summary()




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_28 (InputLayer)        (None, 1, 1, 2048)        0         
_________________________________________________________________
global_max_pooling2d_32 (Glo (None, 2048)              0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
[[0.381931   0.0566115  0.05444313 ... 0.18831639 0.4700506  0.774913  ]]


In [0]:
print(model.predict(inp).shape)

(1, 2048)


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving train.zip to train.zip
User uploaded file "train.zip" with length 71899790 bytes


In [0]:
ls

sample_data/  submission.csv  test.zip  train_relationships.csv  train.zip


In [0]:
# importing required modules 
from zipfile import ZipFile 

# specifying the zip file name 
file_name = "../content/train.zip"

# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zipm: 
	# printing all the contents of the zip file 
  #zipm.printdir() 

	# extracting all the files 
	print('Extracting all the files now...') 
	zipm.extractall("../content/train") 
	print('Done!') 


Extracting all the files now...
Done!


In [0]:
ls

sample_data/    test/     train/                   train.zip
submission.csv  test.zip  train_relationships.csv


In [0]:
from collections import defaultdict
from glob import glob
from random import choice, sample

import cv2
import numpy as np
import pandas as pd
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Input, Dense, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract
from keras.models import Model
from keras.optimizers import Adam

In [0]:
train_file_path = "../content/train_relationships.csv"
train_folders_path = "../content/train/"
val_famillies = "F08"

In [0]:
all_images = glob(train_folders_path + "*/*/*.jpg")

In [0]:
len(all_images)

12379

In [0]:
all_images[0]

'../content/train/F0874/MID3/P09232_face2.jpg'

In [0]:
train_images = [x for x in all_images if val_famillies not in x]

In [0]:
val_images = [x for x in all_images if val_famillies in x]

In [0]:
len(val_images)

1328

In [0]:
val_images[-5:]

['../content/train/F0846/MID2/P08933_face2.jpg',
 '../content/train/F0846/MID4/P08937_face2.jpg',
 '../content/train/F0846/MID4/P08942_face1.jpg',
 '../content/train/F0846/MID4/P08941_face1.jpg',
 '../content/train/F0846/MID4/P08936_face1.jpg']

In [0]:
train_person_to_images_map = defaultdict(list)

In [0]:
ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]

In [0]:
for x in train_images:
    train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

In [0]:
train_person_to_images_map.keys()

dict_keys(['F0058/MID3', 'F0058/MID5', 'F0058/MID1', 'F0058/MID6', 'F0058/MID4', 'F0113/MID3', 'F0113/MID5', 'F0113/MID1', 'F0113/MID2', 'F0113/MID4', 'F0771/MID3', 'F0771/MID5', 'F0771/MID1', 'F0771/MID6', 'F0771/MID2', 'F0771/MID4', 'F0675/MID3', 'F0675/MID5', 'F0675/MID1', 'F0675/MID6', 'F0675/MID2', 'F0675/MID4', 'F0589/MID3', 'F0589/MID2', 'F0589/MID4', 'F0909/MID3', 'F0909/MID1', 'F0909/MID2', 'F0909/MID4', 'F0233/MID3', 'F0233/MID5', 'F0233/MID1', 'F0233/MID6', 'F0233/MID2', 'F0683/MID3', 'F0683/MID5', 'F0683/MID1', 'F0683/MID6', 'F0683/MID2', 'F0683/MID4', 'F0161/MID3', 'F0161/MID1', 'F0161/MID2', 'F0593/MID3', 'F0593/MID5', 'F0593/MID1', 'F0593/MID2', 'F0593/MID4', 'F0531/MID3', 'F0531/MID1', 'F0531/MID2', 'F0531/MID4', 'F0630/MID3', 'F0630/MID1', 'F0630/MID2', 'F0440/MID3', 'F0440/MID1', 'F0440/MID2', 'F0440/MID4', 'F0693/MID3', 'F0693/MID1', 'F0693/MID2', 'F0693/MID4', 'F0408/MID3', 'F0408/MID1', 'F0408/MID2', 'F0408/MID4', 'F0579/MID3', 'F0579/MID5', 'F0579/MID1', 'F0579/MI

In [0]:
len(train_person_to_images_map.keys())

2051

In [0]:
val_person_to_images_map = defaultdict(list)

In [0]:
for x in val_images:
    val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)


In [0]:
len(val_person_to_images_map )

265

In [0]:
p=set(ppl)

In [0]:
len(p)

2316

In [0]:
relationships = pd.read_csv(train_file_path)
relationships = list(zip(relationships.p1.values, relationships.p2.values))
relationships = [x for x in relationships if x[0] in ppl and x[1] in ppl]

In [0]:
relationships[1]

('F0002/MID2', 'F0002/MID3')

In [0]:
len(relationships)

3362

In [0]:
train = [x for x in relationships if val_famillies not in x[0]]
val = [x for x in relationships if val_famillies in x[0]]

In [0]:
def read_img(path):
    img = cv2.imread(path)
    img = np.array(img).astype(np.float)
    return preprocess_input(img, version=2)

In [0]:
def gen(list_tuples, person_to_images_map, batch_size=16):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)
        while len(batch_tuples) < batch_size:
            p1 = choice(ppl)
            p2 = choice(ppl)

            if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
                batch_tuples.append((p1, p2))
                labels.append(0)

        for x in batch_tuples:
            if not len(person_to_images_map[x[0]]):
                print(x[0])

        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
        X1 = np.array([read_img(x) for x in X1])

        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
        X2 = np.array([read_img(x) for x in X2])

        yield [X1, X2], labels

In [0]:
def baseline_model():
    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    base_model = VGGFace(model='resnet50', include_top=False)

    for x in base_model.layers[:]:
        x.trainable = True

    x1 = base_model(input_1)
    x2 = base_model(input_2)

    # x1_ = Reshape(target_shape=(7*7, 2048))(x1)
    # x2_ = Reshape(target_shape=(7*7, 2048))(x2)
    #
    # x_dot = Dot(axes=[2, 2], normalize=True)([x1_, x2_])
    # x_dot = Flatten()(x_dot)

    x1 = Concatenate(axis=-1)([GlobalMaxPool2D()(x1), GlobalAvgPool2D()(x1)])
    x2 = Concatenate(axis=-1)([GlobalMaxPool2D()(x2), GlobalAvgPool2D()(x2)])

    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])

    x = Multiply()([x1, x2])

    x = Concatenate(axis=-1)([x, x3])

    x = Dense(100, activation="relu")(x)
    x = Dropout(0.1)(x)
    out = Dense(1, activation="sigmoid")(x)

    model = Model([input_1, input_2], out)

    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))

    model.summary()

    return model

In [0]:
file_path = "vgg_face_weight.h5"

In [0]:
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [0]:
reduce_on_plateau = ReduceLROnPlateau(monitor="val_loss", mode="min", factor=0.1, patience=20, verbose=1)

In [0]:
callbacks_list = [checkpoint, reduce_on_plateau]

In [0]:
model = baseline_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
vggface_resnet50 (Model)        multiple             23561152    input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
global_max_pooling2d_5 (GlobalM (None, 2048)         0           vggface_resnet50[1][0]           
__________

In [0]:
model.fit_generator(gen(train, train_person_to_images_map, batch_size=16), use_multiprocessing=True,
                    validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=25, verbose=2,
                    workers=4, callbacks=callbacks_list, steps_per_epoch=200, validation_steps=100)

/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:44: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/25


/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:272: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


 - 207s - loss: 3.3803 - acc: 0.5447 - val_loss: 3.8153 - val_acc: 0.5319

Epoch 00001: val_loss improved from inf to 3.81527, saving model to vgg_face_weight.h5
Epoch 2/25
 - 186s - loss: 1.9717 - acc: 0.5969 - val_loss: 2.2034 - val_acc: 0.5825

Epoch 00002: val_loss improved from 3.81527 to 2.20337, saving model to vgg_face_weight.h5
Epoch 3/25
 - 187s - loss: 1.0793 - acc: 0.6272 - val_loss: 1.3290 - val_acc: 0.6000

Epoch 00003: val_loss improved from 2.20337 to 1.32901, saving model to vgg_face_weight.h5
Epoch 4/25
 - 187s - loss: 0.7764 - acc: 0.6544 - val_loss: 0.8824 - val_acc: 0.6275

Epoch 00004: val_loss improved from 1.32901 to 0.88238, saving model to vgg_face_weight.h5
Epoch 5/25
 - 188s - loss: 0.5877 - acc: 0.7006 - val_loss: 0.7714 - val_acc: 0.6444

Epoch 00005: val_loss improved from 0.88238 to 0.77136, saving model to vgg_face_weight.h5
Epoch 6/25


In [0]:
test_path = "../content/test/"

In [0]:
pwd

'/content'

In [0]:
def chunker(seq, size=32):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [0]:
from tqdm import tqdm

submission = pd.read_csv('../content/sample_submission.csv')

In [0]:
predictions = []

In [0]:
for batch in tqdm(chunker(submission.img_pair.values)):
    X1 = [x.split("-")[0] for x in batch]
    X1 = np.array([read_img(test_path + x) for x in X1])

    X2 = [x.split("-")[1] for x in batch]
    X2 = np.array([read_img(test_path + x) for x in X2])

    pred = model.predict([X1, X2]).ravel().tolist()
    predictions += pred


166it [01:07,  1.14it/s]


In [0]:
submission['is_related'] = predictions

submission.to_csv("vgg_face_f.csv", index=False)